In [2]:
from numpy import genfromtxt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import codecs
import numpy as np
import os
import shutil
import torch
import yaml
from datetime import datetime
from torch.utils.data.sampler import RandomSampler
from torch.autograd import Variable


In [3]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [4]:
train = data.iloc[0:1611]
test = data[1611:]
print(train.shape)
print(test.shape)

(1611, 27)
(378, 27)


In [5]:
trainheadlines = data.iloc[0:1611, 2:27].values
trainLabels = data.iloc[0:1611, 1].values
testheadlines = data.iloc[1611:, 2:27].values
testLabels = data.iloc[1611:, 1].values

print(trainheadlines.shape)
print(trainLabels[:10])

(1611, 25)
[0 1 0 0 1 1 0 0 1 1]


In [8]:
trainVocab = []
for row in range(0,len(train.index)):
    trainVocab.append(' '.join(str(x) for x in train.iloc[row,2:27]))

vectorizer = CountVectorizer(min_df = 3)
basictrain = vectorizer.fit_transform(trainVocab)
print(basictrain.shape)

(1611, 14073)


In [6]:
# trainheadlines = []
# trainLabels = []
# for row in range(0,len(train.index)):
#     trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
#     trainLabels.append(train.iloc[row,1])

# basicvectorizer = CountVectorizer()
# basictrain = basicvectorizer.fit_transform(trainheadlines)
# print(basictrain.shape)

# testheadlines = []
# testLabels = []
# for row in range(0,len(test.index)):
#     testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
#     testLabels.append(test.iloc[row,1])

# basictest = basicvectorizer.transform(testheadlines)

In [8]:
def transformHeadlines(headlines):
    outputHeadlines = np.empty([headlines.shape[0], 5])
    for i, headline in enumerate(headlines):
        outputHeadline = [vectorizer.vocabulary_[x] if x in vectorizer.vocabulary_ else -1 for x in vectorizer.build_tokenizer()(headline.lower())]
        `
#         outputHeadlines = np.vstack([outputHeadlines, outputHeadline])
#         print(headline)
        outputHeadlines[i] = outputHeadline[:5]
        
    return np.asarray(outputHeadlines)
        
#     print('shape', outputHeadlines.shape)
#     print(len(outputHeadlines))


In [9]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf
class LSTMLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LSTMLayer, self).__init__()
        self.output_dim = output_dim
        self.lstmLayers = []
        for i in range(25):
            self.lstmLayers.append(nn.LSTM(input_dim, output_dim))


    def forward(self, headlines, hiddens = None):
        outputs = []
        new_hiddens = []
        if hiddens is None:
            hiddens = (Variable(torch.zeros(25, 1, 1, self.output_dim)), Variable(torch.zeros(25, 1, 1, self.output_dim)))
#         print("hiddens.size", hiddens[1][0].size())
        for i, headline in enumerate(headlines):
#             print(np.asarray([headline]).type)
#             print(torch.from_numpy(np.asarray([headline])).float())
#             print(Variable(torch.from_numpy(np.asarray([headline]))).view(1,1,1))
            output, (hidden, cell) = self.lstmLayers[i](Variable(torch.from_numpy(np.asarray([headline])).float()).view(1,1,1), (hiddens[0][i], hiddens[1][i]))
            outputs.append(output)
            new_hiddens.append((hidden, output))
        return outputs, new_hiddens




class PredictLayer(nn.Module):
    def __init__(self, input_dim):
        super(PredictLayer, self).__init__()
        # self.output_dim = output_dim
        # self.lstmLayers = []
        # for i in range(25):
        #     self.lstmLayers.append(nn.LSTM(input_dim, output_dim))
        self.input_dim = input_dim
        self.maxPool = nn.MaxPool1d(25)
        self.fc = nn.Linear(input_dim, 2)


    def forward(self, encodedHeadlines):
        permuted = encodedHeadlines.permute(1, 2, 3, 0).view(1, input_dim, -1)
        maxPooled = self.maxPool(permuted)
        return F.log_softmax(maxPooled[0], dim=-1)

In [10]:
lstmLayer = LSTMLayer(1, 30)
predictLayer = PredictLayer(30)

sample = trainheadlines[1]
print(sample[0])
print(vectorizer.build_tokenizer()(sample[0].lower()))
inputToModel = transformHeadlines(sample)
print(inputToModel.shape)
print('here', inputToModel[:,0])
for i in range(10):
#     print("word", word)
    outputs, new_hiddens = lstmLayer(inputToModel[:, i])
#     print(new_hiddens)
log_probs = predictLayer(new_hiddens)
print(log_probs)

b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'
['why', 'wont', 'america', 'and', 'nato', 'help', 'us', 'if', 'they', 'wont', 'help', 'us', 'now', 'why', 'did', 'we', 'help', 'them', 'in', 'iraq']
(25, 5)
here [ 30957.   4899.  15333.  12121.  20134.  30884.  24571.   2170.  30829.
  12120.  24571.   1241.  25658.   9225.  28343.  15133.   8939.  24237.
   5793.  30614.  15134.  24571.  21144.  24571.  26323.]


IndexError: index 5 is out of bounds for axis 1 with size 5